# 1.数据准备

## 1_1.导入模块

In [60]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
from apscheduler.schedulers.background import BackgroundScheduler
import calendar
import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import time
from datetime import timedelta , datetime, timezone
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

## 1_2.连接数据库

In [61]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

## 获取所需数据

In [62]:
def get_data(hour):
    sql1 = f''' -- 订单&风控信息  近10日数据   
    SELECT om.create_time,om.id as order_id ,om.order_number
    ,om.status
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
    when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
    when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
    ,tod.sku_attributes,tod.product_name,tod.new_actual_money
    ,om.user_mobile,tmu.true_name,tmu.id_card_num
    ,top.total_describes
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
    ,cc.name as channel_name         -- 来源渠道
    ,cc.channel_type_id              -- 渠道id
    ,pa.name as activity_name        -- 活动名称
    ,om.merchant_id,om.merchant_name,pa.type
    ,om.order_method, om.activity_id
    , om.order_type, tojo.app_type, tor.update_time, tomt.reason
    from  db_digua_business.t_order  om
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id
    -- 备注信息合并 
    left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
    on om.id = top.order_id 
    -- 渠道名称
    left join db_digua_business.t_channel cc on om.channel = cc.scene 
    -- 活动名称
    left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
    -- 用户信息 
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    -- 商品信息
    left join db_digua_business.t_order_details tod on om.id = tod.order_id
    -- 京东外部订单关联表
    left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
    -- 商家订单转移表
    left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id

    where om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
    and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -1 month )
    and hour(om.create_time)<{hour}
    '''
    df_order = query(sql1)
    df_order = df_order[df_order.type!=4]
    sql_risk = ''' -- risk等级
            select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
            from db_credit.risk
            '''
    df_risk = query(sql_risk)
    sql3 = '''
        SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
        '''
    df_risk_examine = query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

    sql_ra = ''' -- 996强拒表
        select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
        '''
    df_ra = query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
    
    return df_order, df_risk, df_risk_examine, df_re, df_ra

# 2.数据清理

# 2_1.日期处理

In [63]:
def clean_date(df):
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    return df

## 2_2.备注信息处理

In [64]:
def clean_remark(df):
    df = clean_date(df)
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    
    df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    # 渠道归属
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

    return df

## 2_3.订单去重处理

In [65]:
def clean_order_drop(df):
    df = clean_remark(df)
    dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
    }
    df["状态编码"]=df["status2"].map(dict_status_code)
    df.sort_values(by=["下单日期","状态编码"],inplace=True)
    #删除重复单号
    df.drop_duplicates(subset=["order_id"],inplace=True)
    #删除身份证空值行
    df.dropna(subset=["id_card_num"],axis=0,inplace=True)
    #去刷单订单
    df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
    df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
    #删除身份证空值行
    df.dropna(subset=["status2"],axis=0,inplace=True)
    #删除重复订单
    df.drop_duplicates(subset=["order_id"],inplace=True)
    df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)
    df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)
    return df

## 2_4.定义状态处理

In [66]:
def clean_status(df, df_risk, df_re, df_ra):
    df = clean_order_drop(df)
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 
    # 剔除商家数据
    df = clean.drop_merchant(df)
    # 获取节点状态数据
    df = clean.status_node(df)
    # 剔除据量数据
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    return df, df2

# 3.获取近15天进件top15的归属渠道

In [67]:
def get_top15(df, df2, df_risk_examine):
    today = datetime.now()-timedelta(days=15)
    df = df[df.下单日期>=today]
    df2 = df2[df2.下单日期>=today]
    df_top15 = all_models.data_group(df, df2, df_risk_examine, '归属渠道')[['进件数']]
    df_top15 = df_top15.sort_values(by='进件数', ascending=False)[:15].index.to_list()
    return df_top15
top15_list = ['京东渠道','芝麻租物','搜索渠道','单人聊天会话中的小程序消息卡片（分享）','八派信息','派金花','抖音渠道','九州信息','我的小程序入口','支付宝直播','其他渠道场景渠道。','小程序商家消息（服务提醒）','生活号','聚量派','宜品花']

In [68]:
def get_result(df,value):
    now_before = datetime.now()-pd.DateOffset(months=1)
    year, month = now_before.year, now_before.month
    # 获取上个月月天数
    days_in_month = calendar.monthrange(year, month)[1]
    #获取对比数据
    df = df[(df.归属渠道.isin(top15_list))] # 
    df_pivot = pd.pivot_table(df, values=value, columns='归属渠道', index='下单日期', aggfunc='count')
    df_pivot.index = df_pivot.index.astype('str')
    df_pivot_new = df_pivot.copy()
    df_pivot_new.loc['昨日对比',:] = df_pivot.diff(periods=1).iloc[-1]
    df_pivot_new.loc['周环比',:] = df_pivot.diff(periods=7).iloc[-1]
    df_pivot_new.loc['月同比',:] = df_pivot.diff(periods=30+(days_in_month-30)).iloc[-1]
    
    return df_pivot_new[top15_list]

In [69]:
def my_job(hour):
    # 获取数据
    df, df_risk, df_risk_examine, df_re, df_ra = get_data(hour)
    # 数据处理
    df, df2 = clean_status(df, df_risk, df_re, df_ra)
    # 获取进件top15的归属渠道
    # df_top15 = get_top15(df, df2, df_risk_examine)
    # 获取对比数据
    df_pivot_new_qc = get_result(df, 'order_id')
    df_pivot_new_jj = get_result(df, '进件数')
    return df_pivot_new_qc, df_pivot_new_jj

In [49]:
path = r'\\digua\迪瓜租机\002数据监测\1.渠道对比/'
scheduler = BackgroundScheduler()
job = scheduler.add_job(my_job, 'cron', hour=14, minute=1, args=[14])
job1 = scheduler.add_job(my_job, 'cron', hour=18, minute=1, args=[18])
print('定时任务创建完毕...\n正在执行定时任务my_job...')
# 查看是否添加了任务
print(scheduler.get_jobs())
scheduler.start()
# 模拟主程序
try:
    while True:
        next_run_time = job.next_run_time
        next_run_time1 = job1.next_run_time
        if next_run_time:
            now = datetime.now(timezone.utc)
            sleep_duration = (next_run_time - now).total_seconds()
            if sleep_duration > 0:
                time.sleep(sleep_duration)
        elif next_run_time1:
            now = datetime.now(timezone.utc)
            sleep_duration1 = (next_run_time - now).total_seconds()
            if sleep_duration1 > 0:
                time.sleep(sleep_duration)
        else:
            time.sleep(60)  # 如果没有找到下次运行时间，则等待一段时间后重新检查
except (KeyboardInterrupt, SystemExit):
    # 用户按下 Ctrl+C 或系统要求退出时，优雅地关闭调度器
    scheduler.shutdown()

归属渠道,九州信息,京东渠道,八派信息,其他渠道场景渠道。,单人聊天会话中的小程序消息卡片（分享）,宜品花,小程序商家消息（服务提醒）,我的小程序入口,抖音渠道,搜索渠道,支付宝直播,派金花,生活号,聚量派,芝麻租物
下单日期,,,,,,,,,,,,,,,
2025-05-20,12.0,23.0,119.0,9.0,238.0,4.0,77.0,18.0,9.0,688.0,6.0,38.0,2.0,6.0,1067.0
2025-05-21,28.0,82.0,96.0,6.0,240.0,8.0,47.0,12.0,3.0,612.0,4.0,33.0,4.0,3.0,1051.0
2025-05-22,32.0,62.0,96.0,12.0,234.0,8.0,14.0,15.0,11.0,576.0,4.0,29.0,2.0,6.0,1005.0
2025-05-23,28.0,64.0,107.0,4.0,248.0,11.0,14.0,11.0,9.0,601.0,6.0,35.0,3.0,4.0,992.0
2025-05-24,17.0,72.0,97.0,14.0,199.0,10.0,8.0,20.0,10.0,560.0,3.0,16.0,1.0,5.0,960.0
2025-05-25,13.0,50.0,58.0,3.0,164.0,5.0,8.0,14.0,1.0,470.0,13.0,22.0,1.0,1.0,919.0
2025-05-26,17.0,78.0,76.0,13.0,228.0,5.0,14.0,13.0,12.0,629.0,12.0,26.0,3.0,NaN,1189.0
2025-05-27,17.0,72.0,76.0,9.0,189.0,12.0,7.0,14.0,9.0,600.0,11.0,30.0,NaN,5.0,1252.0
2025-05-28,26.0,74.0,73.0,7.0,219.0,7.0,29.0,9.0,7.0,606.0,5.0,17.0,2.0,2.0,1130.0
